# Μια ολοκληρωμένη διαδικασία ταξινόμησης

Το scikit-learn έχει καθιερώσει την ορολογία που περιγράφει με κομψό τρόπο μια ολοκληρωμένη διαδικασία Μachine Learning, στην περίπτωσή που εξετάζουμε μια διαδικασία ταξινόμησης.

## Pipeline, Εκτιμητές και Μετασχηματιστές

Μια διαδικασία ML ή workflow ή pipeline αποτελείται από μια σεiρα μετασχηματιστών πάνω στα χαρακτηριστικά των δεδομένων που τελειώνει σε έναν εκτιμητή.

Οι μετασχηματιστές χρησιμοποιούνται για να κανουν την προεπεξεργασία (μέσω μετασχηματισμού) των δεδομένων. Είδαμε στο προηγούμενο notebook πέντε μετασχηματιστές: την επιλογή χαρακτηριστικών VarianceThreshold, δύο μετασχηματιστές κανονικοποίησης (τον scaler και τον min_max_scaler), τον εξισορροπητή με τυχαία υπερδειγματοληψία RandomOverSampler και την εξαγωγή χαρακτηριστικών PCA. Αν ανατρέξετε στο προηγούμενο notebook θα δείτε ότι κάνουν fit και transform στο train set και transform στο test set.

Οι μετασχηματιστές γενικα έχουν και αυτοί υπερ-παραμέτρους που επηρρεάζουν τη λειτουργία τους: ο VarianceThreshold είχε το κατώτερο κατώφλι διακύμανσης ο PCA τον αριθμό των κύριων συνιστωσών, ενώ ακόμα και οι scaler, min_max_scaler και RandomOverSampler έχουν αλλά δεν τις εξετάσαμε. Όπως έχουμε πει η επιλογή των υπερ-παραμέτρων (όπως το k του kNN) γίνεται μόνο εμπειρικά μέσω διασταυρούμενης επικύρωσης (cross-validation). Οι μετασχηματιστές και οι υπερπαράμετροι τους επιδρούν λοιπόν στη μορφή των δεδομένων.

Στο τέλος του pipeline VarianceThreshold - scaler - RandomOverSampler - PCA βάλαμε τον εκτιμητή - ταξινομητή MultiLayerPerceptron. O MLP έχει και αυτός υπερ-παραμέτρους και μάλιστα έναν πολύ μεγάλο αριθμό (18 για την ακρίβεια): πλήθος και επίπεδα κρυμμένων νευρώνων, συνάρτηση ενεργοποίησης, βελτιστοποίησης κλπ. Στο προηγούμενο notebook χρησιμοποιήσαμε ένα MLP με σταθερές υπερπαραμέτρους, ωστόσο σε μια ολοκληρωμένη διαδικασία ML οι υπερ-παράμετροι (ή κάποιες από τις υπερπαραμέτρους) του ταξινομητή πρέπει και αυτές να βελτιστοποιηθούν με διαδικασία cross-validation. Οι ταξινομητές και οι υπερπαράμετροι τους δεν επιδρούν στη μορφή των δεδομένων όπως οι μετασχηματιστές αλλά έχουν προφανώς επίδραση στην απόδοση του μοντέλου.

## Ορισμός (επιλογή) ενός εκπαιδευμένου μοντέλου εκτιμητή (ταξινομητή)

Μια ολοκληρωμένη διαδικασία pipeline λόγω του ότι τελειώνει σε έναν εκτιμητή μπορεί να θεωρηθεί και συνολικά ως ένας εκτιμητής, με κανένα, με λίγους ή με περισσότερους μετασχηματιστές πριν από τον εκτιμητή. Στο προηγούμενο παράδειγμα είδαμε στην αρχή ένα μοντέλο με μόνο τον εκτιμητή (εφαρμογή του MLP απευθείας στο dataset) και ένα τελικό μοντέλο με τέσσερεις μετασχηματιστές πριν τον εκτιμητή. Ένα εκπαιδευμένο μοντέλο εκτιμητή (ταξινομητή) αποτελείται
- α) απο την αρχιτεκτονική του, δηλαδή τον συνδυασμό μετασχηματιστών και την επιλογή του τελικού εκτιμητή (το pipeline), και 
- β) από τις (βέλτιστες) τιμές των υπερ-παραμέτρων όλων των προηγουμένων που προκύπτουν από το cross-validation. 

<img src="https://github.com/rasbt/pattern_classification/raw/master/Images/supervised_learning_flowchart.png" width="65%">

Το τελικό βελτιστοποιημένο μοντέλο αποτιμάται στα δεδομένα test και χρησιμοποιείται για να κάνει προβλέψεις σε νέα δεδομένα.

ΠΡΟΣΟΧΗ: εκτός από τη διαχείριση τιμών που απουσιάζουν με Imputer, όλοι οι υπόλοιποι μετασχηματιστές βρίσκονται εντός του σχήματος crossvalidation. 

Η μετατροπή κατηγορικών μεταβλητών γίνεται μετά τη διαχείριση τιμών που απουσιάζουν και πριν το crossvalidation (εκτός δλδ).

## Βελτιστοποίηση ύπερ-παραμέτρων (Hyperparameter optimization)

Είδαμε ότι τόσο οι μετασχηματιστές όσο και οι εκτιμητές έχουν υπερ-παραμέτρους που πρέπει να βελτιστοποιηθούν με cross-validation. Εφόσον τόσο οι μετασχηματιστές όσο και ο εκτιμητής αποτελούν μέρος ενός ενιαίου pipeline, για να βρούμε τις βέλτιστες τιμές όλων των υπερ-παραμέτρων μέσω cross-validation θα πρέπει
- α) για κάθε fold του cross-validation, να υπολογίσουμε την απόδοση όλων των πιθανών συνδυασμών υπερ-παραμέτρων μετασχηματιστών και εκτιμητή και 
- β) να επιλέξουμε το συνδυασμό υπερ-παραμέτρων που έχει τον καλύτερο μέσο όρο με βάση κάποια μετρική σε όλα τα folds.


### Αναζήτηση πλέγματος (grid search)

Η απόδοση όλων των πιθανών συνδυασμών υπερ-παραμέτρων μετασχηματιστών και εκτιμητή γίνεται με αναζήτηση πλέγματος (grid search). Ορίζουμε για κάθε παράμετρο ένα πεδίο ορισμού, συνήθως με ελάχιστο, μέγιστο και κάποιο βήμα και φτιάχνουμε ένα πλέγμα με όλους τους πιθανούς συνδυασμούς τιμών των παραμέτρων. Για παράδειγμα:

για κύριες συνιστώσες PCA (transformer) από 5 μέχρι 15 με βήμα 5 και για έναν kNN (estimator) με k από 1 μέχρι 5 με βήμα 2 παίρνουμε το ακόλουθο grid:

|     |       |        |        |
|-----|-------|--------|--------|
|     | PC=5  | PC=10  | PC=15  |
| k=1 | (1,5) | (1,10) | (1,15) |
| k=3 | (3,5) | (3,10) | (3,15) |
| k=5 | (5,5) | (5,10) | (5,15) |

Για κάθε τιμή υπερπαραμέτρων του grid θα πρέπει να υπολογιστεί ο μέσος όρος του εκτιμητή σε όλα τα folds του cross-validation με βάση το metric (πχ F1) και να επιλεχθεί ο καλύτερος συνδυασμός παραμέτρων. Η συγκεκριμένη στρατηγική αναζήτησης των βέλτιστων υπερπαραμέτρων είναι η εξαντλητική αναζήτηση πλέγματος (exhaustive grid search) και είναι προφανώς πολύ ακριβή υπολογιστικά. Υπάρχουν διάφορες τεχνικές για να περιορίζεται η πολυπλοκότητα του grid search, αλλά δεν το αποφεύγουμε γενικά, γιατί οι υπερπαράμετροι είναι ορίσματα των εκτιμητών και δεν μαθαίνονται από την fit.

Συνοψίζοντας, η βελτιστοποίηση των υπερπαραμέτρων απαιτεί 

- έναν εκτιμητή (έναν ταξινομητή)
- τον πεδίο ορισμού των υπερπαραμέτρων
- ένα τρόπο αναζήτησης των πιθανών συνδυασμών τιμών τους πχ grid search
- ένα σχήμα cross-validation πχ 5-fold
- μια μετρική απόδοσης (ή score) πχ F1-macro

Το scikit-learn μας απλοποιεί σε πολύ μεγάλο βαθμό την κατασκευή pipelines και τη βελτιστοποίηση των υπερπαραμέτρων. Θα το δούμε με ένα παράδειγμα.

Θα βελτιστοποιήσουμε με cross-validation και grid search ένα pipeline με προ-επεξεργασία των δεδομένων από transformers και estimator τον kNN.

# MNIST handwritten digits dataset
Ενημερώνουμε τις βιβλιοθήκες μας. Επίσης θα αγνοήσουμε κάποια warnings

In [ ]:
!pip install -U pip
!pip install -U scikit-learn
!pip install -U numpy
!pip install -U pandas
!pip install -U tensorflow
import warnings 
warnings.filterwarnings('ignore')

Θα εισάγουμε το πολύ γνωστό και απο το Deep Learning dataset [MNIST](http://yann.lecun.com/exdb/mnist/). Θα το εισάγουμε από έναν άλλο server όπου υπάρχει ένα αντίγραφο. Το dataset είναι επίσης διαθέσιμο στο [Kaggle](https://www.kaggle.com/gustavoatt/mnist-original).

In [ ]:
import numpy as np
import pandas as pd

mnist_train = pd.read_csv("https://pjreddie.com/media/files/mnist_train.csv", header=None).values
mnist_test = pd.read_csv("https://pjreddie.com/media/files/mnist_test.csv", header=None).values
mnist = np.concatenate((mnist_train, mnist_test), axis=0) # ενώνουμε train και test

In [ ]:
features = mnist[:, 1:]
targets = mnist[:, :1] # τα labels είναι στην πρώτη κολώνα

To MNIST περιλαμβάνει 70000 δείγματα χειρόγραφων ψηφίων μεγέθους 28x28 pixels, με ετικέτες από το 0 ως το 9. Τα 28x28 pixels κάθε δείγματος αντιστοιχούν σε 768 χαρακτηριστικά με τιμές του γκρι από 0 (μάυρο) εώς 256 (λευκό)

In [ ]:
print(features.shape)
print(targets.shape)
print(np.unique(targets)) #τυπώνουμε τις μοναδικές ετικέτες των labels

Θα κάνουμε οπτικοποίηση ενός δείγματος:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
pixels = features[1500]
pixels = pixels.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

Επειδή το dataset είναι μεγάλο (ειδικά για τον kNN) και για το παράδειγμά μας θέλουμε να δουλέψουμε με λιγότερα δείγματα. Το ανακατεύουμε και παίρνουμε ένα μικρό αριθμό samples

In [ ]:
from sklearn.utils import shuffle
sdata, starget = shuffle(features, targets, random_state=341976)
samples = 1000
data = sdata[0:samples-1,:]
target = starget[0:samples-1]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=20176)

In [ ]:
from sklearn import neighbors
from sklearn.metrics import classification_report
clf = neighbors.KNeighborsClassifier()
clf.fit(X_train,y_train)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

# Pipelines

Για την κατασκευή του μοντέλου θα βασιστούμε στην κλάση Pipeline. Επειδή οι κλάσεις εξισορρόπησης του imblearn όπως η [RandomOverSampler](http://contrib.scikit-learn.org/imbalanced-learn/stable/generated/imblearn.over_sampling.RandomOverSampler.html) τυπικά δεν έχουν μέθοδο transform (έχουν fit_sample) η built-in Pipeline του scikit (from sklearn.pipeline import Pipeline) δεν τις δέχεται ως transformers. Θα φέρουμε την Pipeline από το imblearn (που έχει transform για τους samplers).

In [ ]:
!pip install --upgrade imbalanced-learn

In [ ]:
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline

# φέρνουμε τις γνωστές μας κλάσεις για preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler # φέρνουμε τον StandarScaler ως transformer που έχει .transform kai ΄όχι ως scale()
from imblearn.over_sampling import RandomOverSampler
from sklearn.decomposition import PCA

# αρχικοποιούμε τον εκτιμητή (ταξινομητής) και τους μετασχηματιστές χωρίς υπερ-παραμέτρους
selector = VarianceThreshold()
scaler = StandardScaler()
ros = RandomOverSampler()
pca = PCA()
clf = neighbors.KNeighborsClassifier(n_jobs=-1) # η παράμετρος n_jobs = 1 χρησιμοποιεί όλους τους πυρήνες του υπολογιστή
pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('sampler', ros), ('pca', pca), ('kNN', clf)])

Το pipeline συμπεριφέρεται ως ένας ενιαίος estimator. Μπορούμε να εφαρμόσουμε fit και predict.

In [ ]:
pipe.fit(X_train,y_train)
preds = pipe.predict(X_test)
print(classification_report(y_test, preds))

# GridsearchCV

Στη συνέχεια θα χρησιμποιήσουμε την [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) για να βελτιστοποιήσουμε τις υπερπαραμέτρους μας. Η GridSearchCV κάνει μαζί cross-validation και grid search. Αρχικά μελετάμε το variance των μεταβλητών για τη variance threshold:

In [ ]:
train_variance = X_train.var(axis=0)
print(train_variance)
print(np.max(train_variance))

 Την εισάγουμε και θέτουμε τις τιμές ορισμού των υπερπαραμέτρων:

In [ ]:
from sklearn.model_selection import GridSearchCV

vthreshold = [0, 4000, 8000, 12000] #προσαρμόζουμε τις τιμές μας στο variance που παρατηρήσαμε
n_components = [10, 20, 30, 40, 50, 60]
k = [1, 6, 11, 21, 31, 41] # η υπερπαράμετρος του ταξινομητή

Επειδή ο χώρος αναζήτησης των βέλτιστων υπερπαραμέτρων αρχίζει να μεγαλώνει, ξαναορίζουμε την pipeline με την παράμετρο 'memory': για κάθε fold του crossvalidation και για καθε συνδυασμό υπερπαραμέτρων μετασχηματιστών, τα δεδομένα χρειάζεται να μετασχηματιστούν μία φορά και όχι για κάθε νέα τιμή υπερπαραμέτρων του εκτιμητή. 

Είναι πιθανό στο fit να σας εμφανιστούν κάποια warnings με τη χρήση του memory. Ξανατρέξτε το block του κώδικα.

In [ ]:
pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('sampler', ros), ('pca', pca), ('kNN', clf)], memory = 'tmp')

Μπορούμε να θέτουμε τιμές στις υπερπαραμέτρους των pipelines χρησιμοποιώντας τα ονόματα των estimators, "\_\_", το όνομα της υπερπαραμέτρου, "=" και τις τιμές που της δίνουμε στο grid search. Επίσης μπορούμε να θέσουμε τη μετρική της απόδοσης με την παράμετρο "scoring". Με την παράμετρο "cv" ορίζουμε τον αριθμό των folds. Για βελτιστοποίηση, μπορούμε να θέσουμε την παράμετρο n_jobs=-1 ώστε να χρησιμοποιούνται όλοι οι πυρήνες του υπολογιστή (το default είναι 1).

In [ ]:
estimator = GridSearchCV(pipe, dict(selector__threshold=vthreshold, pca__n_components=n_components, kNN__n_neighbors=k), cv=5, scoring='f1_macro', n_jobs=-1)

Το GridSearchCV είναι επίσης ένας estimator με fit και predict. Ανάλογα το search space η εκτέλεση του  GridSearchCV μπορεί να πάρει αρκετό χρόνο

In [ ]:
import time
start_time = time.time()
estimator.fit(X_train, y_train)
preds = estimator.predict(X_test)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time() - start_time))
print(classification_report(y_test, preds))

Tυπώνουμε τον καλύτερο estimator και τον καλύτερο συνδυασμό υπερπαραμέτρων:

In [ ]:
print(estimator.best_estimator_)
print(estimator.best_params_)

Η στοχαστικότητα στη διαδικασία της ταξινόμησης οφείλεται στα διαφορετικό διαχωρισμό σε folds σε κάθε run αλλά στον RandomOverSampler που επιλέγει τυχαία δείγματα κατά τη δειγματοληψία.

## Επιλογή αρχιτεκτονικής μοντέλου pipeline

Προσοχή, η βέλτιστη αρχιτεκτονική δεν είναι δεδομένη αλλά εξαρτάται από το dataset. Δοκιμαστε στο ίδιο grid, χωρίς scaler και sampler

In [ ]:
pipe = Pipeline(steps=[('selector', selector),('pca', pca), ('kNN', clf)], memory = 'tmp')
estimator = GridSearchCV(pipe, dict(selector__threshold=vthreshold, pca__n_components=n_components, kNN__n_neighbors=k), cv=3, scoring='f1_macro', n_jobs=-1)
# ο estimator με βελτιστοποιημένες υπερπαραμέτρους είναι έτοιμος να κάνει prediction.
# Ωστόσο για να μην πάνε χαμένα δεδομένα (ούτε ένα fold), τον κάνουμε fit σε όλα τα δεδομένα train.
estimator.fit(X_train, y_train)
preds = estimator.predict(X_test)
print(classification_report(y_test, preds))
print(estimator.best_estimator_)
print(estimator.best_params_)

Και φυσικά θα μπορούσαμε να έχουμε διαφορετικά αποτελέσματα εφαρμόζοντας min max scaler αντι standard scaler, undersampling αντί oversampling κοκ. Προφανώς η πιο σημαντική απόφαση στην αρχιτεκτονική του ταξινομητή είναι η επιλογή του τελικού estimator,  αν πχ βάλουμε MLP ή SVM αντί kNN, και βέβαια η βελτιστοποίηση των υπερπαραμέτρων τους.

## Progressive grid search

Στο πεδίο ορισμού των παραμέτρων, ξεκινάμε με μεγάλα διαστήματα και σχετικά λίγα βήματα. Αν διαπιστώσουμε ότι υπαρχει μια περιοχή τιμών κάποιας παραμέτρου που δίνει καλη απόδοση μπορούμε να μικρύνουμε το διάστημα του grid search γύρω της και να βάλουμε περισσότερα βήματα.

In [ ]:
vthreshold = [0]
n_components = [39, 40, 41]
k = [1, 3]
estimator = GridSearchCV(pipe, dict(selector__threshold=vthreshold, pca__n_components=n_components, kNN__n_neighbors=k), scoring='f1_macro', n_jobs=-1)
estimator.fit(X_train, y_train)
preds = estimator.predict(X_test)
print(classification_report(y_test, preds))
print(estimator.best_estimator_)
print(estimator.best_params_)

Το περισσότερο fine grained grid search, αν δώσει καλύτερες τιμές θα έχει βελτιστοποιήσει τον εκτιμητή, αν όχι, τουλάχιστον θα επιβεβαιώσει ότι είμαστε σε ένα καλό τοπικό μέγιστο της συνάρτησης αξιολόγησης.



# Βιβλιοθήκες βελτιστοποίησης υπερπαραμέτρων

Εκτός του scikit-learn, και για μεγάλα και απαιτητικά datasets υπάρχουν πολλές εξειδικευμένες βιβλιοθήκες για τη βελτιστοποίηση των υπερπαραμέτρων. Οι βιβλιοθήκες αυτές χρησιμοποιούν μεθοδολογίες όπως παραλληλισμό, έξυπνη αναζήτηση του χώρου των υπερπαραμέτρων, οπτικοποίηση της διαδικασίας, υποστήριξη βιβλιοθηκών βαθιάς μάθησης κ.α. Οι τεχνικές αναζήτησης στον χώρο των υπερπαραμέτρων είναι αυτόνομο και ευρύ ερευνητικό αντικείμενο.
Θα δώσουμε μερικά παραδείγματα χρήσης τέτοιων βιβλιοθηκών αλλά βέβαια πρέπει κάποιος να μελετήσει αναλυτικά την τεκμηρίωσή τους για να εκμεταλλευτεί πλήρως τις δυνατότητές τους.

## Ray

In [ ]:
#@title
%%html
<a href="https://www.ray.io/" target="_blank"><img src="https://docs.ray.io/en/latest/_static/ray_logo.png" style="width: 300px;"/></a>

In [ ]:
!pip install -U ray 
!pip install -U ray[tune]
!pip install -U tune-sklearn

Στο επόμενο παράδειγμα χρησιμοποιούμε έναν στοχαστικό γραμμικό ταξινομητή το elasticnet που είναι όπως η γραμμική παλινδρόμηση αλλά με όρους ομαλοποίησης. Το στοχαστικό οφείλεται στο ότι κάνουμε υπολογισμό της κλίσης και ανανέωση των βαρών δείγμα-δείγμα όχι για όλα τα δείγματα μαζί όπως είναι η vanilla GD. 

Εξαιτίας αυτού, η εκπαίδευση στοχαστικών ταξινομητών είναι πολύ πιο αργή, καθώς δεν κινούμαστε με βέλτιστο τρόπο προς το τοπικό ελάχιστο. Ωστόσο, το όφελος είναι ότι χρησιμοποιούμε πολύ λιγότερη μνήμη καθώς δεν φορτώνουμε το σύνολο του dataset αλλά μόνο ένα δείγμα.

![](https://miro.medium.com/max/1154/1*jjcOf5V66UUHNfuniT4z-g.png)


Το penalty elasticnet κάνει ομαλοποίηση με ένα συνδυασμό κόστους πρωτοβάθμιου αθροίσματος των βαρών (l1) και δευτεροβάθμιου (l2). H υπερπαράμετρος *l1_ratio* ελέγχει την μίξη των δύο όρων.

Θα χρησιμοποιήσουμε επίσης την συνάρτηση [make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html) για να φτίαξουμε γρήγορα ένα συνθετικό dataset.

Οι υπερπαράμετροι που εξετάζουμε είναι το $α$ που ελέγχει τον βαθμό της ομαλοποιήσης (μεγαλύτερο $α$ μεγαλύτερη ομαλοποίηση) και *l1_ratio*

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(loss='log', penalty='elasticnet')

from sklearn.datasets import make_classification
import numpy as np

# Create dataset
X, y = make_classification(
    n_samples=11000,
    n_features=1000,
    n_informative=50,
    n_redundant=0,
    n_classes=10,
    class_sep=2.5)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1000)

# Example parameters to tune from SGDClassifier
parameter_grid = {"alpha": [1e-4, 1e-1], "l1_ratio": [0.1, 0.2]}

Η παράμετρος early_stopping μας επιτρέπει να τερματίσουμε γρήγορα διατάξεις που δεν υπόσχονται πολλά αν θέσουμε early_stopping=True. max_iters είναι ο μέγιστος αριθμός επαναλήψεων που μπορεί να τρέξει ένα σύνολο υπερπαραμέτρω. Μπορεί να τρέξει για λιγότερες αν είναι early_stopped.

Σημειώστε ότι αυτή είναι η πιο απλή μορφή grid search του ray. Υπάρχουν πολλές περισσότερες στην τεκμηρίωση.

In [ ]:
# from sklearn.model_selection import GridSearchCV
from tune_sklearn import TuneGridSearchCV

tune_search = TuneGridSearchCV(
    sgd, parameter_grid, early_stopping=True, max_iters=5)

import time  # Just to compare fit times
start = time.time()
tune_search.fit(x_train, y_train)
end = time.time()
print("Tune GridSearch Fit Time:", end - start)

Ας κάνουμε μια σύγκριση με το vanilla scikit-learn (αργό, Θα κάνει περίπου 10 λεπτά)

In [ ]:
# n_jobs=-1 enables use of all cores like Tune does
sklearn_search = GridSearchCV(sgd, parameter_grid, n_jobs=-1)

start = time.time()
sklearn_search.fit(x_train, y_train)
end = time.time()
print("Sklearn Fit Time:", end - start)

## Optuna

In [ ]:
#@title
%%html
<a href="https://optuna.readthedocs.io/en/stable/" target="_blank"><img src="https://raw.githubusercontent.com/optuna/optuna/master/docs/image/optuna-logo.png" style="width: 300px;"/></a>

In [ ]:
!pip install optuna

Θα χρησιμοποίησουμε τη βιβλιοθήκη optuna για να βρούμε τη βέλτιστη αρχιτεκτονική (layers, neurons) ενός MLP στο MNIST. (Αργό, περίπου 17-18 λεπτά).

In [ ]:
import optuna
import sklearn.datasets
from sklearn.datasets import fetch_openml
import sklearn.neural_network

def objective(trial):

    n_layers = trial.suggest_int('n_layers', 1, 2)
    layers = []
    for i in range(n_layers):
        layers.append(trial.suggest_int(f'n_units_{i}', 10, 20))
   
    mnist = fetch_openml('mnist_784')
    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        mnist.data, mnist.target)


    clf = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=tuple(layers))
    clf.fit(x_train, y_train)

    return clf.score(x_test, y_test)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

Σε περίπτωση όπου δύο αρχιτεκτονικές έχουν ίδια ή πολύ κοντινή επίδοση, επιλέγουμε την απλούστερη (ξυράφι του Occam). Τρέξτε το επόμενο κελί για να δείτε το σχετικό βίντεο για το ξυράφι του Occam και το overfiting.


In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('Q_AclBHCaUo', width=800, height=300)

Όπως και για την Ray αυτό είναι ένα πολύ περιορισμένο παράδειγμα των δυνατοτήτων της βιβλιοθήκης. Συμβουλευτείτε την τεκμηρίωσή της.

## Άλλα frameworks βελτιστοποίησης

*   [Hyperopt](https://github.com/hyperopt/hyperopt)
*   [mlmachine](https://github.com/petersontylerd/mlmachine)
*   [Polyaxon](https://polyaxon.com/docs/automation/optimization-engine/)
*   [BayesianOptimization](https://github.com/fmfn/BayesianOptimization)
*   [Talos](https://github.com/autonomio/talos)
*   [SHERPA](https://parameter-sherpa.readthedocs.io/en/latest/)
*   [Scikit-Optimize](https://scikit-optimize.github.io/stable/user_guide.html)